LRNG with 10 degrees where c4=c5=c8=c10= 1 andc1=c2=c3=c6=c7=c9= 0.

In [2]:
from collections import deque

class PRNG:
    
    def decimalToBinary(self, n):
        return bin(n).replace("0b","")
    
    def string_to_bitstring(self, string):
        byte_arr = list()
        for i in string:
            if i == '1':
                byte_arr.append(1)
            else:
                byte_arr.append(0)
        binary_format = bytearray(byte_arr)
        return binary_format

    def __init__(self, seed = 0):
        """
        Sets up the internal state of the prng with a seed modulus 1024
        """
        seed = seed % 1024
        bit_string = self.decimalToBinary(seed)
        zero_string = [0 for i in range(10)]
        for i in range(len(bit_string)):
            if bit_string[i] == '1':
                zero_string[i] = 1
        self.internal_state = deque(list(zero_string))
    
    def get_random_bits(self, number = 4):
        output = ''
        for _ in range(number):
            output = output + str(self.internal_state[9])
            first_xor = self.internal_state[7] ^ self.internal_state[9]
            second_xor = first_xor ^ self.internal_state[4]
            third_xor = second_xor ^ self.internal_state[3]
            self.internal_state.rotate()
            self.internal_state[0] = third_xor
        output =  self.von_neumann_pp(output)
        return self.string_to_bitstring(output)
    
    
    def von_neumann_pp(self, inputstring):
        it = iter(inputstring)
        outputstring = ''
        for x in it:
            res = (x, next(it))
            if res == ('0', '0') or res == ('1', '1'):
                continue
            else:
                outputstring = outputstring + res[0]
        return outputstring
                

Monobit test execution.

In [3]:
#!/usr/bin/env python
# retrieved from https://github.com/dj-on-github/sp800_22_tests/blob/master/sp800_22_monobit_test.py
# sp800_22_monobit_test.py
# 
# Copyright (C) 2017 David Johnston
# This program is distributed under the terms of the GNU General Public License.
# 
# This file is part of sp800_22_tests.
# 
# sp800_22_tests is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
# 
# sp800_22_tests is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
# 
# You should have received a copy of the GNU General Public License
# along with sp800_22_tests.  If not, see <http://www.gnu.org/licenses/>.

from __future__ import print_function

import math

def count_ones_zeroes(bits):
    ones = 0
    zeroes = 0
    for bit in bits:
        if (bit == 1):
            ones += 1
        else:
            zeroes += 1
    return (zeroes,ones)

def monobit_test(bits):
    n = len(bits)
    
    zeroes,ones = count_ones_zeroes(bits)
    s = abs(ones-zeroes)
    print("  Ones count   = %d" % ones)
    print("  Zeroes count = %d" % zeroes)
    
    p = math.erfc(float(s)/(math.sqrt(float(n)) * math.sqrt(2.0)))
    
    success = (p >= 0.01)
    return (success,p,None)


In [4]:
prng = PRNG(899)
for seq_length in range(0, 100000, 10000):
    if seq_length == 0:
        continue
    sequence = prng.get_random_bits(seq_length)
    res = monobit_test(sequence)
    print(f'Sequence length: {seq_length}, test passed: {res[0]}, p-value: {res[1]}')
    

  Ones count   = 1244
  Zeroes count = 1255
Sequence length: 10000, test passed: True, p-value: 0.8258368770550711
  Ones count   = 2503
  Zeroes count = 2507
Sequence length: 20000, test passed: True, p-value: 0.9549338894528443
  Ones count   = 3760
  Zeroes count = 3747
Sequence length: 30000, test passed: True, p-value: 0.88073221507181
  Ones count   = 4997
  Zeroes count = 5007
Sequence length: 40000, test passed: True, p-value: 0.9203601988018654
  Ones count   = 6266
  Zeroes count = 6252
Sequence length: 50000, test passed: True, p-value: 0.9004208346710295
  Ones count   = 7500
  Zeroes count = 7514
Sequence length: 60000, test passed: True, p-value: 0.9090346911349301
  Ones count   = 8755
  Zeroes count = 8754
Sequence length: 70000, test passed: True, p-value: 0.9939701673904325
  Ones count   = 10018
  Zeroes count = 10007
Sequence length: 80000, test passed: True, p-value: 0.9380403018231305
  Ones count   = 11259
  Zeroes count = 11263
Sequence length: 90000, test passe